In [ ]:
%matplotlib inline
import adaptive
import matplotlib.pyplot as plt
import pycqed as pq
import numpy as np
from pycqed.measurement import measurement_control
import pycqed.measurement.detector_functions as det
from qcodes import station
station = station.Station()


## Setting up the mock device

Measurements are controlled through the `MeasurementControl` usually instantiated as `MC`

In [ ]:
from pycqed.instrument_drivers.virtual_instruments.mock_device import Mock_Device
MC = measurement_control.MeasurementControl('MC',live_plot_enabled=True, verbose=True)
MC.station = station
station.add_component(MC)

mock_device = Mock_Device('mock_device')
mock_device.mw_pow(-20)
mock_device.res_freq(7.4e9)


## Measuring a resonator using the conventional method

In [ ]:
mock_device.cw_noise_level(.01)

In [ ]:
freqs = np.arange(7.4e9-1e6, 7.4e9+1e6, 10e3)

d = det.Function_Detector(mock_device.S21,value_names=['Magn', 'Phase'], value_units=['V', 'deg'])
MC.set_sweep_function(mock_device.mw_freq)
MC.set_sweep_points(freqs)
MC.set_detector_function(d)
dat=MC.run('test')

In [ ]:
freqs = np.arange(7.4e9-1e6, 7.4e9+1e6, 10e3)

mock_device.acq_type('magn')
d = det.Function_Detector(mock_device.S21,value_names=['Magn'], value_units=['V'])
MC.set_sweep_function(mock_device.mw_freq)
MC.set_sweep_points(freqs)
MC.set_detector_function(d)
dat=MC.run('test')

## Using adaptive sampling

learner = adaptive.Learner1D(f, bounds=(-1, 1))

In [ ]:
def my_func(f):
    mock_device.mw_freq(f)
    return mock_device.S21()

In [ ]:
mock_device.cw_noise_level(0.001)

In [ ]:
adaptive.notebook_extension()
learner = adaptive.Learner1D(my_func, bounds=(7.39e9, 7.41e9))
# The end condition is when the "loss" is less than 0.1. In the context of the
# 1D learner this means that we will resolve features in 'func' with width 0.1 or wider.
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.005)
runner.live_info()

In [ ]:
runner.live_plot(update_interval=1)

## Two D learner

In [ ]:
def my_func(fpow):
    f, pow = fpow
    mock_device.mw_freq(f)
    mock_device.mw_pow(pow)
    return mock_device.S21()

In [ ]:
mock_device.cw_noise_level(0.004)

In [ ]:
adaptive.notebook_extension()
learner = adaptive.Learner2D(my_func, bounds=((7.35e9, 7.45e9), (-30, -10)))
# The end condition is when the "loss" is less than 0.1. In the context of the
# 1D learner this means that we will resolve features in 'func' with width 0.1 or wider.
# learner = adaptive.Learner2D(my_func, bounds=[(-1, 1), (-1, 1)])
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.0005)
runner.live_info()

In [ ]:
def plot(learner):
    plot = learner.plot(tri_alpha=0.2)
    title = f'loss={learner._loss:.3f}, n_points={learner.npoints}'
    return (plot.Image
            + plot.EdgePaths.I.opts(plot=dict(title_format=title))
            + plot)

runner.live_plot(plotter=plot, update_interval=15)